**08-01 합성곱과 풀링**

**08-02 CNN으로 MNIST 분류하기**

In [ ]:
# 1번 레이어 : 합성곱층
# 2번 레이어 : 합성곱층
# 3번 레이어 : 전결합층
# - 3번레이어 특성맵을 펼치고 전결합층(뉴런10개) + 활성화 함수 Softmax

In [1]:
import torch
import torch.nn as nn

inputs = torch.Tensor(1,1,28,28)
print('텐서의 크기: {}'.format(inputs.shape))

텐서의 크기: torch.Size([1, 1, 28, 28])


In [4]:
conv1 = nn.Conv2d(1, 32, 3, padding = 1)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [5]:
conv2 = nn.Conv2d(32, 64, kernel_size =3, padding =1)

In [6]:
pool = nn.MaxPool2d(2)
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [7]:
out = conv1(inputs)
print(out.shape)

torch.Size([1, 32, 28, 28])


In [8]:
out = pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


In [9]:
out = conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


In [10]:
out = pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


In [16]:
out.size(3)
out = out.view(out.size(0), -1)
print(out.shape)

torch.Size([1, 3136])


In [17]:
fc = nn.Linear(3136, 10)
out = fc(out)
print(out.shape)

torch.Size([1, 10])


In [20]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

#device 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

#파라미터 설정
learning_rate = 0.001
training_epochs = 15
batch_size = 100

#데이터로더를 위한 데이터셋 정의
mnist_train = dsets.MNIST(root ='MNIST_data/', train=True, transform=transforms.ToTensor(), download = True)
mnist_test = dsets.MNIST(root ='MNIST_data/', train=False, transform=transforms.ToTensor(), download = True)

#데이터로더 사용
data_loader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size = batch_size, shuffle = True, drop_last = True)

#모델 설계
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        #첫번째층
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1,32,kernel_size=3, stride =1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size =2, stride=2)
        )
        #두번째층
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size = 3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size =2, stride=2))

        #세번째층 : 전결합층
        self.fc = torch.nn.Linear(7*7*64, 10, bias =True)
        torch.nn.init.xavier_uniform_(self.fc.weight)

    #순전파 : 계산방향으로 진행
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

100%|██████████| 9912422/9912422 [00:00<00:00, 103652161.62it/s]


Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 64331223.49it/s]


Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 31788286.49it/s]


Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 13174639.54it/s]


Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



In [22]:
#모델 정의
model = CNN().to(device)

#비용함수 / 옵티마이저
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

total_batch = len(data_loader)
print('총 배치의 수: {}'.format(total_batch))

총 배치의 수: 600


In [23]:
#모델 훈련

for epoch in range(training_epochs):
    avg_cost = 0

    for X,Y in data_loader: #미니 배치 단위로 꺼내온다. X는 미니배치, Y는 레이블
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch +1, avg_cost))

[Epoch:    1] cost = 0.225327089
[Epoch:    2] cost = 0.0599684566
[Epoch:    3] cost = 0.0436554961
[Epoch:    4] cost = 0.0349602848
[Epoch:    5] cost = 0.0288850404
[Epoch:    6] cost = 0.0244276356
[Epoch:    7] cost = 0.0215660036
[Epoch:    8] cost = 0.0172117408
[Epoch:    9] cost = 0.0146206934
[Epoch:   10] cost = 0.0125997029
[Epoch:   11] cost = 0.0100956224
[Epoch:   12] cost = 0.0096427314
[Epoch:   13] cost = 0.00759610953
[Epoch:   14] cost = 0.00672972435
[Epoch:   15] cost = 0.00592247956


In [24]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9872999787330627


**08-03 깊은 CNN으로 MNIST분류하기**

In [25]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

#device 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

#파라미터 설정
learning_rate = 0.001
training_epochs = 15
batch_size = 100

#데이터로더를 위한 데이터셋 정의
mnist_train = dsets.MNIST(root ='MNIST_data/', train=True, transform=transforms.ToTensor(), download = True)
mnist_test = dsets.MNIST(root ='MNIST_data/', train=False, transform=transforms.ToTensor(), download = True)

#데이터로더 사용
data_loader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size = batch_size, shuffle = True, drop_last = True)

#모델 설계
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.keep_prob = 0.5
        #첫번째층
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1,32,kernel_size=3, stride =1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size =2, stride=2)
        )
        #두번째층
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size = 3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size =2, stride=2))

        #세번째층
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size = 3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size =2, stride=2))
        #전결합층
        self.fc1 = torch.nn.Linear(4*4*128, 625, bias =True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=1-self.keep_prob))
        #다섯번째 층
        self.fc2 = torch.nn.Linear(625, 10, bias = True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    #순전파 : 계산방향으로 진행
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.layer4(out)
        out = self.fc2(out)
        return out

In [30]:
#모델 정의
model2 = CNN().to(device)

#비용함수 / 옵티마이저
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

total_batch = len(data_loader)
print('총 배치의 수: {}'.format(total_batch))

총 배치의 수: 600


In [31]:
#모델 훈련

for epoch in range(training_epochs):
    avg_cost = 0

    for X,Y in data_loader: #미니 배치 단위로 꺼내온다. X는 미니배치, Y는 레이블
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model2(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch +1, avg_cost))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (100x1152 and 2048x625)